# Sonification of temporally triggered sound events
In numerous MIR applications one encounters cases where - for the time being not further specified - events occur at certain temporal positions. 
In this notebook, we illustrate how the **libsoni.tse** module can be used to sonify these events with different methods.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import libfmp.b
import libfmp.c4
import librosa
from IPython import display as ipd

from libsoni.core import tse
from libsoni.util.utils import mix_sonification_and_original, plot_sonify_novelty_beats

Fs= 22050

## Simple scenario: time positions
To start with a simple example, let's create a list of arbitrarily chosen time positions, given in seconds:

In [ ]:
# Define some time positions
time_positions = [0.5, 1.25, 2.5, 2.75]

### Sonification of time positions with clicks
Assuming we want to sonify this list, we can use the function **sonify_tse_click** to generate an audio signal comprising clicks at the corresponding time positions.

In [ ]:
# Sonification using libsoni
sonified_time_positions = tse.sonify_tse_click(time_positions=time_positions)

print('Sonified time positions:')
ipd.display(ipd.Audio(sonified_time_positions, rate=Fs))

The clicks generated within **sonify_tse_click** can be adjusted in order to match the respective use case with the following parameters:
 - **click_pitch**: pitch of the click
 - **click_duration**: duration of the click
 - **click_amplitude**: amplitude of the click
 
Let's say we'd rather have deeper sounding, longer clicks, the above sonification changes as follows:

In [ ]:
# Sonification using libsoni
sample_sonified_time_positions = tse.sonify_tse_click(time_positions=time_positions,
                                                      click_pitch=43,
                                                      click_duration=0.5)

print('Sonified time positions with deeper, longer clicks:')
ipd.display(ipd.Audio(sample_sonified_time_positions, rate=Fs))

### Sonification of time positions with samples (TODO: Snap-Sample noch einchecken!)
Before we get to a real-world audio example, let's stay with the arbitrarily chosen time positions and explain how samples can be used for sonification.
The first thing we need for this is - guess three times - a sample. Why don't we use a finger snap sound?

In [ ]:
# Load sample
#snap_sample, _ = librosa.load(os.path.join('data_audio','samples', 'snap.wav'), sr=Fs)
snap_sample, _ = librosa.load(os.path.join('data_audio','samples', 'snare.wav'), sr=Fs)

print('Snap sample:')
ipd.display(ipd.Audio(snap_sample, rate=Fs))

Now, using the function **sonify_tse_sample**, we can sonify our list of time positions with our sample:

In [ ]:
# Sonification using libsoni
sample_sonified_arbitrarily_chosen_time_positions = tse.sonify_tse_sample(time_positions=time_positions,
                                                                          sample=snap_sample)
                                                 
print('Sonified time positions with snap sample:')
ipd.display(ipd.Audio(sample_sonified_arbitrarily_chosen_time_positions, rate=Fs))

## Scenario 1: Sonifying beat annotations

In [ ]:
# Borodin
title = 'Borodin: String Quartet No. 2, 3rd movement' 
fn_ann = os.path.join('data_csv', 'demo_tse', 'FMP_C6_Audio_Borodin-sec39_RWC_quarter.csv')
fn_wav = os.path.join('data_audio', 'demo_tse', 'FMP_C6_Audio_Borodin-sec39_RWC.wav')

plot_sonify_novelty_beats(fn_wav, fn_ann, title);

borodin_audio, _ = librosa.load(fn_wav, sr=Fs)
borodin_df = pd.read_csv(fn_ann)

# Sonification using libsoni
sonified_borodin = tse.sonify_tse_click(time_positions=borodin_df.to_numpy())

print('Original audio and sonification of beat positions with clicks:')
ipd.display(ipd.Audio(mix_sonification_and_original(sonified_borodin, borodin_audio, panning = 0.5), rate=Fs))

# Chopin
title = 'Chopin: Op.68, No. 3' 
fn_ann = os.path.join('data_csv', 'demo_tse', 'FMP_C6_Audio_Chopin.csv')
fn_wav = os.path.join('data_audio', 'demo_tse', 'FMP_C6_Audio_Chopin.wav')

plot_sonify_novelty_beats(fn_wav, fn_ann, title);

chopin_audio, _ = librosa.load(fn_wav, sr=Fs)
chopin_df = pd.read_csv(fn_ann)

# Sonification using libsoni
castanets_sample, _ = librosa.load(os.path.join('data_audio','samples', 'castanets.wav'), sr=Fs)
sonified_chopin = tse.sonify_tse_sample(time_positions=chopin_df.to_numpy(),
                                        sample=castanets_sample)

print('Original audio and sonification of beat positions with castanet sample:')
ipd.display(ipd.Audio(mix_sonification_and_original(sonified_chopin, chopin_audio, panning = 0.5), rate=Fs))


# Faure
title = 'Fauré: Op.15' 
fn_ann = os.path.join('data_csv', 'demo_tse', 'FMP_C6_Audio_Faure_Op015-01-sec0-12_SMD126.csv')
fn_wav = os.path.join('data_audio', 'demo_tse', 'FMP_C6_Audio_Faure_Op015-01-sec0-12_SMD126.wav')
plot_sonify_novelty_beats(fn_wav, fn_ann, title);

faure_audio, _ = librosa.load(fn_wav, sr=Fs)
metronome_sample, _ = librosa.load(os.path.join('data_audio','samples', 'metronome.wav'), sr=Fs)
faure_df = pd.read_csv(fn_ann)
sonified_faure = tse.sonify_tse_sample(time_positions=faure_df.to_numpy(),sample=metronome_sample)
ipd.display(ipd.Audio(mix_sonification_and_original(sonified_faure, faure_audio, panning = 0.5), rate=Fs))

## Scenario 2: Drumset

In [ ]:
fn_wav = os.path.join('data_audio','demo_tse','FMP_C6_Audio_Queen_AnotherOneBitesTheDust-Beginning.wav')

fn_ann = os.path.join('data_csv','demo_tse','FMP_C6_F01_Queen_drums.csv')
queen_drums_df = pd.read_csv(fn_ann, delimiter =';')
ipd.display(queen_drums_df)


queen_audio, _ = librosa.load(fn_wav, sr=Fs)

kick_times = queen_drums_df[queen_drums_df['label']=='kick']['position'].to_numpy()
hihat_times = queen_drums_df[queen_drums_df['label']=='hihat']['position'].to_numpy()
snare_times = queen_drums_df[queen_drums_df['label']=='snare']['position'].to_numpy()

hihat_sample, _ = librosa.load(os.path.join('data_audio','samples','hi-hat.wav'),sr=Fs)
snare_sample, _ = librosa.load(os.path.join('data_audio','samples','snare.wav'),sr=Fs)
kick_sample, _ = librosa.load(os.path.join('data_audio','samples','bass-drum.wav'),sr=Fs)
    
    
    
queen_drums = [(hihat_times, hihat_sample),
               (snare_times, snare_sample),
               (kick_times, kick_sample)]


sonified = tse.sonify_tse_multiple_samples(queen_drums)


ipd.display(ipd.Audio(sonified,rate=Fs))

ipd.display(ipd.Audio(mix_sonification_and_original(queen_audio, sonified),rate=Fs))

## Scenario 3: Structure Annotations aux *Town Musicians of Bremen*

https://www.audiolabs-erlangen.de/resources/MIR/FMP/C4/C4S1_MusicStructureGeneral.html

Links Text, Erklärung , Rechts Bild
<img src="data/demo_tse/town_musicians_of_bremen.png" alt="town musicians of bremen" width="400" height="400">

In [ ]:
fn_ann = os.path.join('data_csv', 'demo_tse', 'FMP_C4_Audio_Chopin_Op028-11_003_20100611-SMD.csv')
chopin_df = pd.read_csv(fn_ann, delimiter=';')
ann, color_ann = libfmp.c4.read_structure_annotation(fn_ann)
ipd.display(chopin_df)



fn_wav = os.path.join('data_audio', 'demo_tse', 'FMP_C4_Audio_Chopin_Op028-11_003_20100611-SMD.wav')
chopin_audio, _ = librosa.load(fn_wav, sr=Fs)

times_A = chopin_df[chopin_df['label'] == 'A']['start'].to_numpy()
times_B = chopin_df[chopin_df['label'] == 'B']['start'].to_numpy()
times_C = chopin_df[chopin_df['label'] == 'C']['start'].to_numpy()
times_D = chopin_df[chopin_df['label'] == 'D']['start'].to_numpy()

sample_A, _ = librosa.load(os.path.join('data_audio','samples', 'rooster.wav'), sr=Fs)
sample_B, _ = librosa.load(os.path.join('data_audio','samples', 'cat.wav'), sr=Fs)
sample_C, _ = librosa.load(os.path.join('data_audio','samples', 'dog.wav'), sr=Fs)
sample_D, _ = librosa.load(os.path.join('data_audio','samples', 'donkey.wav'), sr=Fs)

collection = [(times_A, sample_A),(times_B, sample_B),(times_C, sample_C),(times_D, sample_D)]

sonification_bremen = tse.sonify_tse_multiple_samples(collection, offset_relative=0.5)

color_ann = {'A': [1, 0, 0, 0.2],'B': [0, 1, 0, 0.2],  'C': [0, 0, 1, 0.2], 'D': [1, 1, 0, 0.2]}


fig, ax = libfmp.b.plot_segments(ann, colors=color_ann, figsize=(6, 1))
plt.xlabel('Time (frames)');
plt.show()

ipd.display(ipd.Audio(mix_sonification_and_original(sonification_bremen, chopin_audio), rate=Fs))